In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import xgboost as xgb
import catboost as cat
from xgboost import XGBClassifier, XGBRegressor
from xgboost import plot_importance
from catboost import CatBoostRegressor
from matplotlib import pyplot
import shap
import re

base_dir = 'data/'

import os
for dirname, _, filenames in os.walk(base_dir):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
from time import time
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from scipy import stats
import lightgbm as lgb
from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.linear_model import LinearRegression
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
from datetime import datetime
import gc
import json
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_columns', 1000)

data/sample_submission.csv
data/reduce_test.csv
data/train_labels.csv
data/test.csv
data/reduce_train.csv
data/train.csv
data/specs.csv


In [2]:
def read_data():
    print('Reading train.csv file....')
    train = pd.read_csv(base_dir + 'train.csv')
    print('Training.csv file have {} rows and {} columns'.format(train.shape[0], train.shape[1]))

    print('Reading test.csv file....')
    test = pd.read_csv(base_dir + 'test.csv')
    print('Test.csv file have {} rows and {} columns'.format(test.shape[0], test.shape[1]))

    print('Reading train_labels.csv file....')
    train_labels = pd.read_csv(base_dir + 'train_labels.csv')
    print('Train_labels.csv file have {} rows and {} columns'.format(train_labels.shape[0], train_labels.shape[1]))

    print('Reading specs.csv file....')
    specs = pd.read_csv(base_dir + 'specs.csv')
    print('Specs.csv file have {} rows and {} columns'.format(specs.shape[0], specs.shape[1]))

    print('Reading sample_submission.csv file....')
    sample_submission = pd.read_csv(base_dir + 'sample_submission.csv')
    print('Sample_submission.csv file have {} rows and {} columns'.format(sample_submission.shape[0], sample_submission.shape[1]))
    return train, test, train_labels, specs, sample_submission

train, test, train_labels, specs, sample_submission = read_data()

Reading train.csv file....
Training.csv file have 11341042 rows and 11 columns
Reading test.csv file....
Test.csv file have 1156414 rows and 11 columns
Reading train_labels.csv file....
Train_labels.csv file have 17690 rows and 7 columns
Reading specs.csv file....
Specs.csv file have 386 rows and 3 columns
Reading sample_submission.csv file....
Sample_submission.csv file have 1000 rows and 2 columns


In [8]:
train_labels[train_labels.installation_id == "0006a69f"]

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.0,0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.5,2
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.0,3


In [23]:
train_0006a69f = train[train.installation_id == "0006a69f"]

In [26]:
Assessment = train_0006a69f[train.title.str.contains('Assessment')]
Assessment

Boolean Series key will be reindexed to match DataFrame index.


,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
2185,3bfd1a65,901acc108f55a5a1,2019-08-06T05:22:01.344Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0006a69f,1,2000,0,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2186,db02c830,901acc108f55a5a1,2019-08-06T05:22:01.400Z,"{""event_count"":2,""game_time"":37,""event_code"":2...",0006a69f,2,2025,37,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2187,a1e4395d,901acc108f55a5a1,2019-08-06T05:22:01.403Z,"{""description"":""Pull three mushrooms out of th...",0006a69f,3,3010,37,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2188,a52b92d5,901acc108f55a5a1,2019-08-06T05:22:05.242Z,"{""description"":""Pull three mushrooms out of th...",0006a69f,4,3110,3901,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2189,a1e4395d,901acc108f55a5a1,2019-08-06T05:22:05.244Z,"{""description"":""To pick a mushroom, pull it ou...",0006a69f,5,3010,3901,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
...,...,...,...,...,...,...,...,...,...,...,...
4151,8fee50e2,a9ef3ecb3d1acc6a,2019-08-06T20:50:30.601Z,"{""hat"":8,""destination"":""middle"",""coordinates"":...",0006a69f,28,4020,31551,Bird Measurer (Assessment),Assessment,TREETOPCITY
4152,a16a373e,a9ef3ecb3d1acc6a,2019-08-06T20:50:32.888Z,"{""coordinates"":{""x"":539,""y"":73,""stage_width"":1...",0006a69f,29,4070,33818,Bird Measurer (Assessment),Assessment,TREETOPCITY
4153,070a5291,a9ef3ecb3d1acc6a,2019-08-06T20:50:33.283Z,"{""correct"":true,""hats"":[4,8,5],""event_count"":3...",0006a69f,30,4100,34209,Bird Measurer (Assessment),Assessment,TREETOPCITY
4154,f6947f54,a9ef3ecb3d1acc6a,2019-08-06T20:50:33.289Z,"{""duration"":17742,""misses"":0,""stage_number"":2,...",0006a69f,31,2030,34209,Bird Measurer (Assessment),Assessment,TREETOPCITY


In [51]:
for i in range(2232-2185):
    print(Assessment['event_data'].iloc[i])

{"version":"1.0","event_count":1,"game_time":0,"event_code":2000}
{"event_count":2,"game_time":37,"event_code":2025}
{"description":"Pull three mushrooms out of the ground and order them from shortest to tallest!","identifier":"Dot_PullMushrooms","media_type":"audio","total_duration":3000,"event_count":3,"game_time":37,"event_code":3010}
{"description":"Pull three mushrooms out of the ground and order them from shortest to tallest!","identifier":"Dot_PullMushrooms","media_type":"audio","duration":3864,"event_count":4,"game_time":3901,"event_code":3110}
{"description":"To pick a mushroom, pull it out of the ground with your finger!","identifier":"Dot_PickFinger","media_type":"audio","total_duration":2680,"event_count":5,"game_time":3901,"event_code":3010}
{"height":4,"coordinates":{"x":329,"y":550,"stage_width":1015,"stage_height":762},"event_count":6,"game_time":6475,"event_code":4025}
{"description":"To pick a mushroom, pull it out of the ground with your finger!","identifier":"Dot_Pi

In [36]:
Assessment[Assessment['game_session'] == '901acc108f55a5a1']

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
2185,3bfd1a65,901acc108f55a5a1,2019-08-06T05:22:01.344Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0006a69f,1,2000,0,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2186,db02c830,901acc108f55a5a1,2019-08-06T05:22:01.400Z,"{""event_count"":2,""game_time"":37,""event_code"":2...",0006a69f,2,2025,37,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2187,a1e4395d,901acc108f55a5a1,2019-08-06T05:22:01.403Z,"{""description"":""Pull three mushrooms out of th...",0006a69f,3,3010,37,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2188,a52b92d5,901acc108f55a5a1,2019-08-06T05:22:05.242Z,"{""description"":""Pull three mushrooms out of th...",0006a69f,4,3110,3901,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2189,a1e4395d,901acc108f55a5a1,2019-08-06T05:22:05.244Z,"{""description"":""To pick a mushroom, pull it ou...",0006a69f,5,3010,3901,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2190,28ed704e,901acc108f55a5a1,2019-08-06T05:22:07.812Z,"{""height"":4,""coordinates"":{""x"":329,""y"":550,""st...",0006a69f,6,4025,6475,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2191,a52b92d5,901acc108f55a5a1,2019-08-06T05:22:07.812Z,"{""description"":""To pick a mushroom, pull it ou...",0006a69f,7,3110,6475,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2192,9d29771f,901acc108f55a5a1,2019-08-06T05:22:07.816Z,"{""description"":""That's one!"",""identifier"":""Dot...",0006a69f,8,3021,6475,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2193,c74f40cd,901acc108f55a5a1,2019-08-06T05:22:08.427Z,"{""description"":""That's one!"",""identifier"":""Dot...",0006a69f,9,3121,7084,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2194,28ed704e,901acc108f55a5a1,2019-08-06T05:22:09.742Z,"{""height"":1,""coordinates"":{""x"":176,""y"":526,""st...",0006a69f,10,4025,8400,Mushroom Sorter (Assessment),Assessment,TREETOPCITY


In [37]:
Assessment[Assessment['game_session'] == '9501794defd84e4d']

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
3697,3bfd1a65,9501794defd84e4d,2019-08-06T20:34:53.812Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0006a69f,1,2000,0,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
3698,db02c830,9501794defd84e4d,2019-08-06T20:34:53.872Z,"{""event_count"":2,""game_time"":67,""event_code"":2...",0006a69f,2,2025,67,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
3699,a1e4395d,9501794defd84e4d,2019-08-06T20:34:53.878Z,"{""description"":""Pull three mushrooms out of th...",0006a69f,3,3010,67,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
3700,28ed704e,9501794defd84e4d,2019-08-06T20:34:56.658Z,"{""height"":5,""coordinates"":{""x"":615,""y"":512,""st...",0006a69f,4,4025,2858,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
3701,a52b92d5,9501794defd84e4d,2019-08-06T20:34:56.660Z,"{""description"":""Pull three mushrooms out of th...",0006a69f,5,3110,2858,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
3702,9d29771f,9501794defd84e4d,2019-08-06T20:34:56.663Z,"{""description"":""That's one!"",""identifier"":""Dot...",0006a69f,6,3021,2858,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
3703,c74f40cd,9501794defd84e4d,2019-08-06T20:34:57.286Z,"{""description"":""That's one!"",""identifier"":""Dot...",0006a69f,7,3121,3483,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
3704,28ed704e,9501794defd84e4d,2019-08-06T20:34:57.686Z,"{""height"":3,""coordinates"":{""x"":799,""y"":465,""st...",0006a69f,8,4025,3891,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
3705,9d29771f,9501794defd84e4d,2019-08-06T20:34:57.687Z,"{""description"":""two..."",""identifier"":""Dot_Two""...",0006a69f,9,3021,3891,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
3706,c74f40cd,9501794defd84e4d,2019-08-06T20:34:58.207Z,"{""description"":""two..."",""identifier"":""Dot_Two""...",0006a69f,10,3121,4408,Mushroom Sorter (Assessment),Assessment,TREETOPCITY


In [38]:
Assessment[Assessment['game_session'] == '6bdf9623adc94d89']


,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
2768,3bfd1a65,6bdf9623adc94d89,2019-08-06T05:37:50.020Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0006a69f,1,2000,0,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2769,db02c830,6bdf9623adc94d89,2019-08-06T05:37:50.078Z,"{""event_count"":2,""game_time"":77,""event_code"":2...",0006a69f,2,2025,77,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2770,a1e4395d,6bdf9623adc94d89,2019-08-06T05:37:50.082Z,"{""description"":""Pull three mushrooms out of th...",0006a69f,3,3010,77,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2771,7da34a02,6bdf9623adc94d89,2019-08-06T05:37:52.799Z,"{""coordinates"":{""x"":199,""y"":484,""stage_width"":...",0006a69f,4,4070,2784,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2772,28ed704e,6bdf9623adc94d89,2019-08-06T05:37:53.631Z,"{""height"":1,""coordinates"":{""x"":171,""y"":519,""st...",0006a69f,5,4025,3625,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2773,a52b92d5,6bdf9623adc94d89,2019-08-06T05:37:53.632Z,"{""description"":""Pull three mushrooms out of th...",0006a69f,6,3110,3625,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2774,9d29771f,6bdf9623adc94d89,2019-08-06T05:37:53.635Z,"{""description"":""That's one!"",""identifier"":""Dot...",0006a69f,7,3021,3625,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2775,c74f40cd,6bdf9623adc94d89,2019-08-06T05:37:54.253Z,"{""description"":""That's one!"",""identifier"":""Dot...",0006a69f,8,3121,4250,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2776,28ed704e,6bdf9623adc94d89,2019-08-06T05:37:54.930Z,"{""height"":2,""coordinates"":{""x"":496,""y"":502,""st...",0006a69f,9,4025,4925,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2777,9d29771f,6bdf9623adc94d89,2019-08-06T05:37:54.934Z,"{""description"":""two..."",""identifier"":""Dot_Two""...",0006a69f,10,3021,4925,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
